In [ ]:
# Importowanie bibliotek
import os, glob, warnings, itertools, pandas as pd, numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Ustawienie katalogu roboczego
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [ ]:
# Ustawienia trenowania i walidacji
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
TRAINING_EPOCHS = 50
TUNNING_EPOCHS = 50
STOP_TRAINING = 50
IMAGE_SIZE = (150, 150)
INPUT_SHAPE = (150, 150, 3)
LAYER = 11
CLASS_MODE = 'categorical'
LOSS_TYPE = 'categorical_crossentropy'
MODEL_NAME = 'men-women_vgg16.keras'
TRAIN_DIR = './men-women/train'
VAL_DIR = './men-women/val'
TEST_DIR = './men-women/test'

In [ ]:
# Funkcja do budowy wykresów trenowania i walidacji
def trainingPlots(tuningHistory):
    global ACC, VAL_ACC, LOSS, VAL_LOSS, STOP_TRAINING
    ACC += tuningHistory.history['acc']
    VAL_ACC += tuningHistory.history['val_acc']
    LOSS += tuningHistory.history['loss']
    VAL_LOSS += tuningHistory.history['val_loss']
    epochs = range(1, len(ACC) + 1)
    plt.figure(1, figsize = (8, 6))
    plt.plot(epochs, ACC, color = 'blue', label = 'Dokładność trenowania', linewidth = 3)
    plt.plot(epochs, VAL_ACC, color = 'red', label = 'Dokładność walidacji', linewidth = 3)
    plt.vlines(STOP_TRAINING, 0, 1, color = 'green', label = 'Początek dostrajania', linewidth = 3)
    plt.legend(loc = 'lower right')
    plt.xlabel('Epoka', fontsize = 20)
    plt.ylabel('Dokładność', fontsize = 20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.legend(fontsize = 20)
    plt.grid()
    plt.show()

    plt.figure(2, figsize = (8, 6))
    plt.plot(epochs, LOSS, color = 'blue', label = 'Strata trenowania', linewidth = 3)
    plt.plot(epochs, VAL_LOSS, color = 'red', label = 'Strata walidacji', linewidth = 3)
    plt.vlines(STOP_TRAINING, 0, 1, color = 'green', label = 'Początek dostrajania', linewidth = 3)
    plt.legend(loc = 'upper right')
    plt.xlabel('Epoka', fontsize = 20)
    plt.ylabel('Strata', fontsize = 20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.legend(fontsize = 20)
    plt.grid()
    plt.show()

In [ ]:
# Definicja generatora danych treningowych
def trainGenerator():
    train_datagen = ImageDataGenerator(
            rescale = 1. / 255,
            rotation_range = 40,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            fill_mode = 'nearest')

    train_generator = train_datagen.flow_from_directory(
            TRAIN_DIR,
            target_size = IMAGE_SIZE,
            batch_size = TRAIN_BATCH_SIZE,
            class_mode = CLASS_MODE,
            shuffle = True)
    return train_generator

In [ ]:
# Definicja generatora danych walidacyjnych
def validationGenerator():
    val_datagen = ImageDataGenerator(rescale = 1. / 255)
    val_generator = val_datagen.flow_from_directory(
            VAL_DIR,
            target_size = IMAGE_SIZE,
            batch_size = VAL_BATCH_SIZE,
            class_mode = CLASS_MODE,
            shuffle = False)
    return val_generator

In [ ]:
# Konfiguracja etapu trenowania
def modelTraining(model, train_generator, val_generator):
    mc = ModelCheckpoint(
        filepath = './training_best_weights.keras',
        monitor = 'val_loss',
        save_best_only = True,
        verbose = 1)
    trainingHistory = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // train_generator.batch_size,
        validation_data = val_generator,
        validation_steps = val_generator.samples // val_generator.batch_size,
        epochs = TRAINING_EPOCHS,
        callbacks = [mc])
    model.load_weights('./training_best_weights.keras')
    trainedModel = model
    os.remove('./training_best_weights.keras')

    global ACC, VAL_ACC, LOSS, VAL_LOSS, STOP_TRAINING
    ACC = trainingHistory.history['acc']
    VAL_ACC = trainingHistory.history['val_acc']
    LOSS = trainingHistory.history['loss']
    VAL_LOSS = trainingHistory.history['val_loss']
    STOP_TRAINING = len(ACC)
    return trainingHistory, trainedModel

In [ ]:
# Konfiguracja etapu dostrajania
def modelTuning(trainedModel, trainingHistory, train_generator, val_generator):
    es = EarlyStopping(
        monitor = 'val_acc',
        patience = 10,
        verbose = 1)
    mc = ModelCheckpoint(
        filepath = './tuning_best_weights.keras',
        monitor = 'val_loss',
        save_best_only = True,
        verbose = 1)
    tuningHistory = trainedModel.fit(
            train_generator,
            steps_per_epoch = train_generator.samples // train_generator.batch_size,
            validation_data = val_generator,
            validation_steps = val_generator.samples // val_generator.batch_size,
            epochs = TRAINING_EPOCHS + TUNNING_EPOCHS,
            initial_epoch = trainingHistory.epoch[-1] + 1,
            callbacks = [es, mc])
    trainedModel.load_weights('./tuning_best_weights.keras')
    tunedModel = trainedModel
    os.remove('./tuning_best_weights.keras')
    return tuningHistory, tunedModel

In [ ]:
# Budowa generatorów danych
train_generator = trainGenerator()
val_generator = validationGenerator()

In [ ]:
# Budowa modelu - dodanie bazy konwolucyjnej
convBase = VGG16(weights = 'imagenet', include_top = False, input_shape = INPUT_SHAPE)
model = Sequential()
model.add(convBase)

In [ ]:
# Budowa modelu - dodanie nowego klasyfikatora
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
# Zamrożenie bazy konwolucyjnej
for layer in convBase.layers:
  layer.trainable = False
model.compile(loss=LOSS_TYPE,
  optimizer = optimizers.RMSprop(2e-5),
  metrics = ['acc'])

In [ ]:
# Uruchomienie trenowania sieci
trainingHistory, trainedModel = modelTraining(
  model,
  train_generator,
  val_generator)

In [ ]:
# Odmrożenie górnych warstw bazy konwolucyjnej
for layer in convBase.layers[LAYER:]:
  layer.trainable = True
trainedModel.compile(loss = LOSS_TYPE,
  optimizer = optimizers.RMSprop(1e-5),
  metrics = ['acc'])

In [ ]:
# Uruchomienie dostrajania sieci
tuningHistory, tunedModel = modelTuning(trainedModel, trainingHistory,
  train_generator, val_generator)
tunedModel.save('./men-women_vgg16.keras')

In [ ]:
# Wyświetlenie wykresów trenowania i walidacji
trainingPlots(tuningHistory)

In [ ]:
# Funkcja do budowy macierzy pomyłek
def plotCM(cm, labels):
    plt.figure(figsize = (4, 4))
    plt.imshow(cm, interpolation = 'nearest', cmap = plt.cm.Blues)
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, horizontalalignment = "center", fontsize = 18)
    plt.yticks(tick_marks, labels, verticalalignment = "center", fontsize = 18)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j]),
            horizontalalignment = "center",
            verticalalignment = "center",
            color = "white" if cm[i, j] > thresh else "black",
            fontsize = 20)
    plt.ylabel('Klasa rzeczywista', fontsize = 18)
    plt.xlabel('Klasa przewidywana', fontsize = 18)
    plt.tight_layout()
    plt.show()

In [ ]:
# Definicja generatora danych testowych
def testGenerator():
    test_datagen = ImageDataGenerator(rescale = 1. / 255)
    test_generator = test_datagen.flow_from_directory(
            TEST_DIR,
            target_size = IMAGE_SIZE,
            batch_size = TEST_BATCH_SIZE,
            class_mode = CLASS_MODE,
            shuffle = False)
    return test_generator

In [ ]:
# Uruchomienie testowania modelu
model = keras.models.load_model('./' + MODEL_NAME)
test_generator = testGenerator()
labels = (test_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())
steps=test_generator.samples//test_generator.batch_size
test_generator.reset()
Y_pred = model.predict(test_generator, steps=steps, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)

In [ ]:
# Wyświetlenie metryk jakości klasyfikacji
LOSS, ACC = model.evaluate(test_generator, steps = steps, verbose = 1)
print("\nDokładność: %f\nStrata: %f\n" % (ACC, LOSS))

In [ ]:
# Wyświetlenie raportu klasyfikacji
print(classification_report(test_generator.classes, y_pred, target_names = list(labels.values())))

In [ ]:
# Wyświetlenie macierzy pomyłek
cnf_matrix = confusion_matrix(test_generator.classes, y_pred)
plotCM(cnf_matrix, list(labels.values()))

In [ ]:
# Funkcja implementująca prognozę klasy nowych obrazów
def classPred(fileList):
  for filePath in fileList:
    org_img = load_img(filePath)
    raw_img = load_img(filePath, target_size = (150, 150))
    img = img_to_array(raw_img)
    img = np.expand_dims(img, axis = 0)
    img /= 255.
    prob = model.predict(img)
    predicted_class = np.argmax(prob)
    labels_list = list(labels.values())
    plt.imshow(org_img)
    plt.title('Prognoza: ' + labels_list[predicted_class])
    plt.axis('off')
    plt.show()

In [ ]:
# Uruchomienie predykcji nowych obrazów
fileList = glob.glob('./men-women/predict/*')
classPred(fileList)